# Agentic AI Kaggle Competition Assistant (Enhanced)

This notebook runs an AI assistant for Kaggle competitions, demonstrating key concepts like **Custom Tools**, **Session Management**, and **Observability**.

**Instructions:**
1. Add the "Meta Kaggle" and "Meta Kaggle Code" datasets.
2. Add your `GOOGLE_API_KEY` as a secret.
3. Run the cells below.

## 1. Setup and Imports

In [ ]:
!pip install -q google-generativeai pandas tqdm python-dotenv beautifulsoup4 requests
import os, re, json, pandas as pd
from urllib.parse import urlparse
from tqdm import tqdm
from datetime import datetime
from typing import Dict, List, Any
from dataclasses import dataclass, field
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool
import requests
from bs4 import BeautifulSoup

GOOGLE_API_KEY = None
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    print("API Key loaded from Kaggle Secrets.")
except (ImportError, Exception):
    print("Could not load from Kaggle Secrets, checking environment variables.")
    from dotenv import load_dotenv
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google AI configured successfully.")
else:
    print("Error: GOOGLE_API_KEY not found.")

## 2. Agent Source Code Modules

In [ ]:
# --- src/built_in_tools.py ---
def web_fetch(prompt: str) -> str:
    url_match = re.search(r"(https?://[^\s]+)", prompt)
    if not url_match:
        return "Error: No valid URL found in the prompt."
    url = url_match.group(0)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        for script_or_style in soup(["script", "style", "header", "footer", "nav"]:
            script_or_style.decompose()
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        clean_text = "\n".join(chunk for chunk in chunks if chunk)
        return clean_text[:10000]
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP Error fetching {url}: {http_err}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

# --- src/kaggle_api.py ---
# ... (contents of kaggle_api.py) ...

# --- src/tools.py ---
datasets = load_data() if GOOGLE_API_KEY else None
# ... (contents of tools.py, importing from the built_in_tools defined above) ...

# --- src/agent.py ---
# ... (contents of agent.py) ...


## 3. Initialize and Run the Agent

In [ ]:
if GOOGLE_API_KEY and datasets:
    agent = KaggleAgent(api_key=GOOGLE_API_KEY)
    print("Kaggle Assistant is ready!")
    # Example of the new workflow
    url = "https://www.kaggle.com/competitions/titanic"
    response = agent.run(f"Give me a summary of the competition at {url}, including top solutions and code.")
    print(f"
Assistant Response:
{response}")
else:
    print("Agent not initialized. Check API key and dataset paths.")